# Bone Fracture Classification Using Custom CNN in PyTorch

In this project, we aim to build a convolutional neural network (CNN) to classify bone fracture images as either fractured or not fractured. The CNN will be implemented using PyTorch, with image data preprocessing, augmentation, model training, and evaluation steps documented below.


## Step 1: Import Libraries

In this step, we import necessary libraries required for building the model, handling image data, and performing analysis. 

- `torch` and `torchvision`: For model building and image processing.
- `matplotlib`: For plotting results.
- `sklearn.metrics`: For model evaluation using metrics like classification report and confusion matrix.


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from PIL import ImageFile

## Step 2: Load and Preprocess Data

We define the directory paths for our training, validation, and test datasets. Then, we define transformations to apply to the images, including resizing, normalization, and data augmentation techniques (like random flipping and rotations for the training set). 

Data augmentation helps improve the model's ability to generalize by introducing variability in the training data.

In [2]:
# Define dataset directories
train_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\train"
val_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\val"
test_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\test"

In [3]:
# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
# Enhanced data augmentation for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224
        transforms.RandomHorizontalFlip(),  # Random horizontal flip for augmentation
        transforms.RandomRotation(10),  # Randomly rotate images by 10 degrees
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random zoom
        transforms.ToTensor(),  # Convert PIL images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

In [5]:
# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms['val'])

## Step 3: Define Data Loaders

Here we create PyTorch `DataLoader` objects to efficiently load and batch our datasets. We also shuffle the training data to introduce randomness in training and provide information on the number of samples in each dataset.

- **Batch Size**: We use a batch size of 32 for training, validation, and testing.
- **Shuffling**: The training data is shuffled to ensure that each mini-batch is different from epoch to epoch.

In [6]:
# Create data loaders with a defined batch size
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
# Display dataset sizes and class names
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

Number of training samples: 9246
Number of validation samples: 829
Number of test samples: 506


In [8]:
classes = os.listdir(train_dir)
print(f"Classes found: {len(classes)}")
classes

Classes found: 2


['fractured', 'not fractured']

## Step 4: Build the Custom CNN Model

In this step, we define our custom CNN architecture using PyTorch. The model includes the following components:

- **Convolutional Layers**: Three convolutional layers to extract features from the input images, with increasing numbers of filters.
- **Batch Normalization**: To stabilize and accelerate training.
- **Max Pooling**: After each convolutional layer, we apply max pooling to down-sample the feature maps.
- **Fully Connected Layers**: Two fully connected layers to learn the higher-level representations and output the final prediction.

The final layer uses a **Sigmoid activation function** for binary classification (fractured vs. not fractured).

In [9]:
# Custom CNN architecture with Batch Normalization and Dropout
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Define convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Input: RGB image
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 256)  # Adjust based on image size after pooling
        self.dropout = nn.Dropout(0.5)  # Dropout for regularization
        self.fc2 = nn.Linear(256, 1)  # Binary classification output

    def forward(self, x):
        # Forward pass through convolutional layers with batch normalization and pooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 28 * 28)  # Flatten the tensor for the fully connected layers
        x = F.relu(self.fc1(x))  # First fully connected layer with ReLU activation
        x = self.dropout(x)  # Apply dropout
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

## Step 5: Set Device, Loss Function, and Optimizer

We move the model to the GPU (if available) to leverage faster training. We define:

- **Loss Function**: `BCELoss` (Binary Cross Entropy Loss) is used because this is a binary classification task.
- **Optimizer**: `Adam` optimizer is used for weight updates, known for its adaptive learning rate and good convergence properties.
- **Learning Rate Scheduler**: A step-based scheduler that reduces the learning rate after every 5 epochs is applied to fine-tune the model.

In [11]:
# Set device for GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model and move it to the specified device
model = CustomCNN().to(device)

In [12]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # For binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Learning rate scheduler

## Step 6: Train the Model

This section contains the core training loop for the CNN. Each epoch consists of:

- **Forward Pass**: Passing the input images through the model.
- **Loss Calculation**: Calculating the binary cross entropy loss.
- **Backpropagation**: Computing gradients and updating weights using the optimizer.
- **Validation**: After each epoch, the model is evaluated on the validation set to monitor progress and prevent overfitting.

We print the training and validation loss after every epoch to track performance over time.

In [13]:
# Training loop function with learning rate scheduling
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        # Training phase
        for images, labels in train_loader:
            images, labels = images.to(device), labels.float().to(device)  # Move data to GPU
            optimizer.zero_grad()  # Zero gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs.view(-1), labels)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
            running_loss += loss.item()  # Accumulate loss

        # Adjust learning rate
        scheduler.step()

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():  # Disable gradient calculation
            for images, labels in val_loader:
                images, labels = images.to(device), labels.float().to(device)
                outputs = model(images)
                loss = criterion(outputs.view(-1), labels)
                val_loss += loss.item()

        # Print epoch results
        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"Train Loss: {running_loss / len(train_loader):.4f}, "
              f"Val Loss: {val_loss / len(val_loader):.4f}")

In [ ]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

c:\Users\daans\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10], Train Loss: 49.6550, Val Loss: 40.5048
Epoch [2/10], Train Loss: 49.8162, Val Loss: 40.5048


## Step 7: Evaluate the Model

After training, we evaluate the model on the test dataset. The following metrics are used for evaluation:

- **Classification Report**: Shows precision, recall, and F1-score for each class.
- **Confusion Matrix**: Provides insights into the number of true positives, true negatives, false positives, and false negatives.
- **AUC-ROC Score**: This metric evaluates the ability of the model to distinguish between classes, which is especially useful in binary classification tasks.

In [ ]:
# Function to evaluate the model on the test set and calculate metrics
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = (outputs.view(-1) > 0.5).float()  # Convert probabilities to binary predictions
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Classification report
    print(classification_report(all_labels, all_preds, target_names=classes))

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # AUC-ROC Score
    auc_score = roc_auc_score(all_labels, all_preds)
    print(f"AUC-ROC Score: {auc_score:.4f}")

In [ ]:
# Evaluate the model
evaluate_model(model, test_loader)

## Step 8: Analyze Model Performance

Here, we analyze the model's performance based on the test evaluation metrics. We discuss:

- The confusion matrix to identify any potential issues with class imbalance.
- The AUC-ROC score to measure how well the model can separate the fractured and non-fractured images.
- The classification report to examine the precision, recall, and F1-score for each class.

## Step 9: Next Steps and Improvements

- **Hyperparameter Tuning**: Experimenting with different batch sizes, learning rates, and optimizers.
- **Early Stopping**: Implementing early stopping to prevent overfitting during training.
- **Data Augmentation**: Adding more aggressive augmentation techniques like random brightness or contrast adjustments.
- **Transfer Learning**: Exploring transfer learning using pre-trained models like ResNet to improve performance.